In [1]:
pip install --upgrade pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import lit
from pyspark.sql.functions import count
from pyspark.sql.functions import col
import random

In [3]:
# Criando uma SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Exercicio Intro') \
    .getOrCreate()

###Nesta etapa, adicione código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

In [4]:
df_nomes = spark.read.csv("nomes_aleatorios.txt")
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



###Como não informamos no momento da leitura do arquivo, o Spark não identificou o Schema por padrão e definiu todas as colunas como string. Para ver o Schema, use o método df_nomes.printSchema(). Nesta etapa, será necessário adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe.

In [5]:
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [6]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.printSchema()

root
 |-- Nomes: string (nullable = true)



In [7]:
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



### Ao dataframe (df_nomes), adicione nova coluna chamada Escolaridade e atribua para cada linha um dos três valores de forma aleatória: Fundamental, Medio ou Superior.

In [8]:
# Função que recebe elementos de uma lista e os adiciona em uma nova coluna do dataframe, de forma aleatória
def randomizar_elementos(dataframe,lista_elementos,nome_coluna):
  udf_elemento_aleatorio = spark.udf.register('udf_elemento_aleatorio', lambda x:random.choice(lista_elementos))
  dataframe = dataframe.withColumn(nome_coluna,udf_elemento_aleatorio(lit(0)))
  return dataframe

In [9]:
# Lista com os níveis de escolaridade a serem adicionados
escolaridade = ['Fundamental','Medio','Superior']
# Adicionando a coluna 'Escolaridade' ao dataframe por meio da função randomizar_elementos
df_nomes = randomizar_elementos(df_nomes,escolaridade,'Escolaridade')

df_nomes.show(5)

+----------------+------------+
|           Nomes|Escolaridade|
+----------------+------------+
|  Frances Bennet|       Medio|
|   Jamie Russell|    Superior|
|  Edward Kistler| Fundamental|
|   Sheila Maurer|    Superior|
|Donald Golightly|    Superior|
+----------------+------------+
only showing top 5 rows



###Ao dataframe (df_nomes), adicione nova coluna chamada Pais e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória.

In [10]:
# Lista dos países da América do Sul a serem adicionados
paises_america_sul = ['Argentina','Bolívia','Brasil','Chile','Colômbia','Equador','Guiana','Paraguai','Peru','Suriname','Uruguai','Venezuela']
# Adicionando a coluna 'Pais' ao dataframe por meio da função randomizar_elementos
df_nomes = randomizar_elementos(df_nomes,paises_america_sul,'Pais')

df_nomes.show(5)

+----------------+------------+--------+
|           Nomes|Escolaridade|    Pais|
+----------------+------------+--------+
|  Frances Bennet|    Superior|Suriname|
|   Jamie Russell|    Superior|Paraguai|
|  Edward Kistler|       Medio|  Guiana|
|   Sheila Maurer|    Superior|  Guiana|
|Donald Golightly|       Medio|Paraguai|
+----------------+------------+--------+
only showing top 5 rows



### Ao dataframe (df_nomes), adicione nova coluna chamada AnoNascimento e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória.

In [11]:
# Obtendo o número de linhas do dataframe 'df_nomes'
num_linhas = df_nomes.select(count('*')).first()[0]
# Lista de anos aleatórios entre 1945 e 2010
anos_aleatorios = [random.randint(1945, 2010) for _ in range(num_linhas)]
# Adicionando a coluna 'AnoNascimento' por meio da função randomizar_elementos
df_nomes = randomizar_elementos(df_nomes, anos_aleatorios, "AnoNascimento")

df_nomes.show(5)

+----------------+------------+---------+-------------+
|           Nomes|Escolaridade|     Pais|AnoNascimento|
+----------------+------------+---------+-------------+
|  Frances Bennet|    Superior|Venezuela|         1986|
|   Jamie Russell| Fundamental| Suriname|         1951|
|  Edward Kistler|       Medio|  Uruguai|         1959|
|   Sheila Maurer|       Medio|    Chile|         1998|
|Donald Golightly| Fundamental|Argentina|         1961|
+----------------+------------+---------+-------------+
only showing top 5 rows



### Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século. Armazene o resultado em outro dataframe chamado df_select e mostre 10 nomes deste.

In [36]:
# Convertendo a coluna "AnoNascimento" para tipo inteiro
df_nomes = df_nomes.withColumn("AnoNascimento", df_nomes["AnoNascimento"].cast("int"))

# Filtrando os registros de pessoas que nasceram neste século e armazenando o resultado em um novo dataframe
df_select = df_nomes.select("Nomes","AnoNascimento")
df_select = df_select.filter(df_select.AnoNascimento >= 2000)

df_select.show(10)

+----------------+-------------+
|           Nomes|AnoNascimento|
+----------------+-------------+
|   Sheila Maurer|         1956|
| Howard Lazarine|         1964|
|  Lorenzo Woodis|         1963|
|     Page Marthe|         1988|
|  Herbert Morris|         1994|
|     Sandra Todd|         1999|
|   Milton Dillon|         1967|
|   Lynne Dustman|         1947|
|Richelle Vasquez|         1960|
| Pamela Banuelos|         1987|
+----------------+-------------+
only showing top 10 rows



### Usando Spark SQL repita o processo da Pergunta 6.

In [ ]:
# Criando uma tabela temporária chamada 'pessoas' para execução das consultas com Spark SQL
df_nomes.createOrReplaceTempView('pessoas')

In [18]:
# Executando consulta SQL para selecionas as pessoas que nasceram neste século
nascimento_filtrado_sql = spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000')

nascimento_filtrado_sql.show(10)

+--------------+------------+---------+-------------+
|         Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------+------------+---------+-------------+
|Frances Bennet|    Superior| Paraguai|         2010|
|   Paul Kriese| Fundamental| Colômbia|         1998|
|Kara Mcelwaine|       Medio|   Brasil|         2007|
|  Leroy Strahl| Fundamental|Argentina|         1956|
|Lorenzo Woodis|    Superior| Suriname|         1973|
|Herbert Morris|       Medio|Venezuela|         1983|
|   Albert Leef| Fundamental|Venezuela|         1967|
|  Rebecca Snow| Fundamental|   Brasil|         1964|
|Gabriel Colyer| Fundamental| Paraguai|         1967|
|      Mary Lee|    Superior|Venezuela|         1959|
+--------------+------------+---------+-------------+
only showing top 10 rows



### Usando o método select do Dataframe df_nomes, Conte o número de pessoas que são da geração Millennials (nascidos entre 1980 e 1994) no Dataset

In [20]:
# Filtrando as pessoas que nasceram entre 1980 e 1994
df_millennials = df_nomes.filter((col('AnoNascimento') >= 1980) & (col('AnoNascimento') <= 1994))

# Contando o número de pessoas da geração Millennials
count_millennials = df_millennials.count()

print(f"Número de pessoas da geração Millennials: {count_millennials}")

Número de pessoas da geração Millennials: 2273205


### Repita o processo da Pergunta 8 utilizando Spark SQL

In [21]:
# Executando consulta SQL para contar o número de pessoas da geração Millennials
resultado_sql = spark.sql("SELECT COUNT(*) AS count_millennials FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994")

# Extraindo o resultado da consulta SQL
count_millennials_sql = resultado_sql.first()["count_millennials"]

print(f"Número de pessoas da geração Millennials: {count_millennials}")

Número de pessoas da geração Millennials: 2273205


In [24]:
# Criando uma função para gerar a consulta SQL de contagem por geração
def consulta_geracao(geracao,ano_inicial,ano_final):
  query = f"""
      SELECT Pais, '{geracao}' AS Geracao, COUNT(*) AS Quantidade
      FROM pessoas
      WHERE AnoNascimento BETWEEN {ano_inicial} AND {ano_final}
      GROUP BY Pais
  """
  return query

# Informações de cada geração
geracoes = [
    {"nome": "Baby Boomers", "ano_inicio": 1944, "ano_fim": 1964},
    {"nome": "Geração X", "ano_inicio": 1965, "ano_fim": 1979},
    {"nome": "Millennials", "ano_inicio": 1980, "ano_fim": 1994},
    {"nome": "Geração Z", "ano_inicio": 1995, "ano_fim": 2015}
]

# Consultando os resultados para cada geração e armazenando em uma lista
resultados_geracoes = []
for geracao_info in geracoes:
    geracao_query = consulta_geracao(geracao_info["nome"],geracao_info["ano_inicio"],geracao_info["ano_fim"])
    resultado = spark.sql(geracao_query)
    resultados_geracoes.append(resultado)

# Combinando os DataFrames usando union
df_resultado = resultados_geracoes[0]
for i in range(1, len(resultados_geracoes)):
    df_resultado = df_resultado.union(resultados_geracoes[i])

# Ordenando o resultado por Pais, Geração e Quantidade
df_resultado = df_resultado.orderBy("Pais", "Geracao", "Quantidade")

df_resultado.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    252122|
|Argentina|   Geração X|    189529|
|Argentina|   Geração Z|    202364|
|Argentina| Millennials|    188772|
|  Bolívia|Baby Boomers|    252152|
|  Bolívia|   Geração X|    189465|
|  Bolívia|   Geração Z|    202466|
|  Bolívia| Millennials|    189217|
|   Brasil|Baby Boomers|    252029|
|   Brasil|   Geração X|    189497|
|   Brasil|   Geração Z|    202352|
|   Brasil| Millennials|    189583|
|    Chile|Baby Boomers|    253018|
|    Chile|   Geração X|    189495|
|    Chile|   Geração Z|    201949|
|    Chile| Millennials|    189826|
| Colômbia|Baby Boomers|    251718|
| Colômbia|   Geração X|    189632|
| Colômbia|   Geração Z|    201791|
| Colômbia| Millennials|    189532|
+---------+------------+----------+
only showing top 20 rows

